In [1]:
import os
import posixpath
import xarray as xr
import rioxarray
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as opt
import pandas as pd
import json

# parallel computing
from dask.distributed import Client

# personnal packages
import optim.space_rejection as sr
import geometry.internal_orientation as gio
import geometry.external_orientation as geo

In [2]:
images_root = 'D:/OneDrive/Documents/Cours/4A/SFE/data/KH-5_ARGON_images'
products = []

for x in os.listdir(images_root):
    if os.path.isdir(posixpath.join(images_root, x)):
        products.append(x)
        
products

['DS09034A007MC018',
 'DS09034A007MC019',
 'DS09034A008MC019',
 'DS09034A008MC020',
 'DS09034A008MC021',
 'DS09034A008MC022',
 'DS09058A024MC012',
 'DS09058A024MC013']

In [3]:
all_params_path = posixpath.join(images_root, "images_params.json")
if os.path.exists(posixpath.join(images_root, "images_params.json")) == False:
    all_params = {
        "description": "Internal and external orientation parameters for KH-5 ARGON images"
    }
    with open(all_params_path, "w") as f:
        json.dump(all_params, f, indent=4)

In [4]:
rasters = [
    'DS09058A024MC012',
    'DS09058A024MC013'
]

all_GCPs = []

for i_raster, raster in enumerate(rasters):
    # raster_path = posixpath.join(images_root, raster, raster + '_a.tif')
    GCPs_path = posixpath.join(images_root, raster, 'GCP.points')
    GCPs_i = pd.read_csv(GCPs_path, encoding="windows-1252", skiprows=1)
    GCPs_i.columns = ["x_gr", "y_gr", "x_img", "y_img", "enable", "dX", "dY", "residual"]
    GCPs_i.drop(columns=["enable", "dX", "dY", "residual"], inplace=True)
    GCPs_i.loc[:, "y_img"] = - GCPs_i.loc[:, "y_img"]
    GCPs_i.loc[:, "image"] = raster
    all_GCPs.append(GCPs_i)
    
GCPs = pd.concat(all_GCPs, ignore_index=True)
print(len(GCPs), "GCPs found for", len(rasters), "rasters")
GCPs.head()

331 GCPs found for 2 rasters


,x_gr,y_gr,x_img,y_img,image
0,515164.992968,8.691528e+06,10090.231823,6378.932688,DS09058A024MC012
1,518689.505642,8.687275e+06,10106.594368,6190.345399,DS09058A024MC012
2,521658.229779,8.685388e+06,10083.782353,6076.763062,DS09058A024MC012
3,523413.708207,8.683043e+06,10101.936417,5980.737615,DS09058A024MC012
4,528288.719248,8.683017e+06,9995.579875,5857.839376,DS09058A024MC012


In [5]:
DEM_root = 'D:/OneDrive/Documents/Cours/4A/SFE/data/NPI_DEMs/NP_S0_DTM20'
file = posixpath.join(DEM_root, "S0_DTM20.tif")

DEM = rioxarray.open_rasterio(file, chunks=True)
DEM = DEM.rename({"x": "x_gr", "y": "y_gr"})
DEM.rio.set_nodata(0., inplace=True)
DEM = DEM.where(DEM < 3e38, 0.)
DEM

<xarray.DataArray (band: 1, y_gr: 36248, x_gr: 22205)> Size: 3GB
dask.array<where, shape=(1, 36248, 22205), dtype=float32, chunksize=(1, 5760, 5760), chunktype=numpy.ndarray>
Coordinates:
  * band         (band) int32 4B 1
  * x_gr         (x_gr) float64 178kB 4.022e+05 4.022e+05 ... 8.463e+05
  * y_gr         (y_gr) float64 290kB 8.978e+06 8.978e+06 ... 8.254e+06
    spatial_ref  int32 4B 0
Attributes: (12/14)
    DataType:                Generic
    AREA_OR_POINT:           Area
    RepresentationType:      ATHEMATIC
    STATISTICS_COVARIANCES:  34445.33087394282
    STATISTICS_MAXIMUM:      1711.7961425781
    STATISTICS_MEAN:         69.648349688442
    ...                      ...
    STATISTICS_SKIPFACTORX:  1
    STATISTICS_SKIPFACTORY:  1
    STATISTICS_STDDEV:       185.59453352387
    _FillValue:              3.4e+38
    scale_factor:            1.0
    add_offset:              0.0

In [9]:
h = DEM.interp(y_gr=("z", GCPs.y_gr.values), x_gr=("z", GCPs.x_gr.values), method="nearest").compute()
GCPs.loc[:, "z_gr"] = h.values.reshape(-1)
GCPs = GCPs.loc[:, ["image", "x_gr", "y_gr", "z_gr", "x_img", "y_img"]]
GCPs.head()

,image,x_gr,y_gr,z_gr,x_img,y_img
0,DS09058A024MC012,515164.992968,8.691528e+06,475.462158,10090.231823,6378.932688
1,DS09058A024MC012,518689.505642,8.687275e+06,901.544189,10106.594368,6190.345399
2,DS09058A024MC012,521658.229779,8.685388e+06,857.430908,10083.782353,6076.763062
3,DS09058A024MC012,523413.708207,8.683043e+06,848.491821,10101.936417,5980.737615
4,DS09058A024MC012,528288.719248,8.683017e+06,1077.836304,9995.579875,5857.839376


In [10]:
for i_raster, raster in enumerate(rasters):
    with open(all_params_path, "r") as f:
        all_params = json.load(f)
    
    # fiducial coordinates
    xc, yc, alpha, delta_xi, delta_eta = all_params[raster]["internal_orientation"]["xc"], all_params[raster]["internal_orientation"]["yc"], all_params[raster]["internal_orientation"]["alpha"] * np.pi / 180, all_params[raster]["internal_orientation"]["delta_xi"], all_params[raster]["internal_orientation"]["delta_eta"]
    x_img, y_img = GCPs.loc[GCPs.image == raster, "x_img"], GCPs.loc[GCPs.image == raster, "y_img"]
    xi, eta = gio.image_to_fiducial_coordinates(x_img.values, y_img.values, xc, yc, alpha, delta_eta, delta_xi)
    GCPs.loc[GCPs.image == raster, ["xi", "eta"]] = np.array([xi, eta]).T
    
GCPs.head()

,image,x_gr,y_gr,z_gr,x_img,y_img,xi,eta
0,DS09058A024MC012,515164.992968,8.691528e+06,475.462158,10090.231823,6378.932688,0.064344,0.041918
1,DS09058A024MC012,518689.505642,8.687275e+06,901.544189,10106.594368,6190.345399,0.064449,0.040679
2,DS09058A024MC012,521658.229779,8.685388e+06,857.430908,10083.782353,6076.763062,0.064303,0.039932
3,DS09058A024MC012,523413.708207,8.683043e+06,848.491821,10101.936417,5980.737615,0.064419,0.039301
4,DS09058A024MC012,528288.719248,8.683017e+06,1077.836304,9995.579875,5857.839376,0.063741,0.038494


In [12]:
f = 76.2e-3
# bounds = (
#     sr.pack_parameters(
#             pp=[127.e-3/2 - 20., 127.e-3/2 - 20.],
#             ld_coeffs=[-10., -10., -10., -10., -10., -10.],
#             eo_params=[[-50.e3, -50.e3, 2.5e5, -20.*np.pi/180, -20.*np.pi/180, -np.inf][i] for _ in range(len(rasters)) for i in range(6)],
#             n_img=len(rasters)
#         ), 
#     sr.pack_parameters(
#             pp=[127.e-3/2 + 20., 127.e-3/2 + 20.],
#             ld_coeffs=[10., 10., 10., 10., 10., 10.],
#             eo_params=[[50.e3, 50.e3, 4.5e5, 20.*np.pi/180, 20.*np.pi/180, np.inf][i] for _ in range(len(rasters)) for i in range(6)],
#             n_img=len(rasters)
#         ),
# )

res = opt.least_squares(
    sr.space_rejection,
    x0=sr.pack_parameters(
        pp=[127.e-3/2, 127.e-3/2],
        ld_coeffs=[0., 0., 0., 0., 0., 0.],
        eo_params=[462640,8751087, 3.22e5, 0., 0., -135 *np.pi/180, 578380,8641266, 3.22e5, 0., 0., -135 *np.pi/180],
        n_img=len(rasters)
    ),
    args=(GCPs, f, len(rasters)),
    # method="trf",
    x_scale="jac",
    # jac="3-point",
    max_nfev=1000,
    # bounds=bounds,
)
print(res)

if res.success:
    pp, ld_coeffs, eo_params = sr.unpack_parameters(res.x, len(rasters))
    print(pp)
    print(ld_coeffs)
    print(eo_params)

     message: The maximum number of function evaluations is exceeded.
     success: False
      status: 0
         fun: [ 2.498e-03]
           x: [ 3.275e-02  9.435e-02 ...  1.245e-01 -1.486e+00]
        cost: 3.118781485014954e-06
         jac: [[ 6.225e-02 -7.860e-02 ...  6.888e-04 -3.539e-05]]
        grad: [ 1.555e-04 -1.963e-04 ...  1.720e-06 -8.838e-08]
  optimality: 0.00019631073348533605
 active_mask: [ 0.000e+00  0.000e+00 ...  0.000e+00  0.000e+00]
        nfev: 1000
        njev: 983
